In [4]:
%run dataset.ipynb

In [5]:
class Office31Dataset(Dataset):
    @property
    def base(self):
        return super(Office31Dataset, self)

In [6]:
def office31_init(self, resolution=[100,100], input_shape=[-1]):
    self.base.__init__('office31', 'dual_select')
    path = '../data_nn/office31'
    domain_names = list_dir(path)
    #domain_names = 'amazon', 'dslr', 'webcam'
    images = []
    didxs, oidxs = [], [] #domain / office

    for dx, dname in enumerate(domain_names):
        domainpath = os.path.join(path, dname, 'images')
        object_names = list_dir(domainpath)
        for ox, oname in enumerate(object_names):
            objectpath = os.path.join(domainpath, oname)
            filenames = list_dir(objectpath)
            for fname in filenames:
                if fname[-4:] != '.jpg':
                    continue
                imagepath = os.path.join(objectpath, fname)
                pixels = load_image_pixels(imagepath, resolution, input_shape)
                images.append(pixels)
                didxs.append(dx)
                oidxs.append(ox)

    self.image_shape = resolution + [3]
    xs = np.asarray(images, np.float32)

    ys0 = onehot(didxs, len(domain_names))
    ys1 = onehot(oidxs, len(object_names))
    ys = np.hstack([ys0, ys1])

    self.shuffle_data(xs, ys, 0.8)
    self.target_names = [domain_names, object_names]
    self.cnts = [len(domain_names)]

In [7]:
def office31_forward_postproc(self, output, y):
    #예측과 실제 정답을 구분하여 저장하는과정
    #np.hsplits(나누고자 하는 것, 몇개로)나누는 함수
    outputs, ys = np.hsplit(output, self.cnts), np.hsplit(y, self.cnts)

    #dual_select이 없기 때문에 'select'을 활용
    loss0, aux0 = self.base.forward_postproc(outputs[0], ys[0], 'select')
    loss1, aux1 = self.base.forward_postproc(outputs[1], ys[1], 'select')

    #손실 합산
    return loss0 + loss1, [aux0, aux1]

In [8]:
def office31_backprop_postproc(self, G_loss, aux):
    aux0, aux1 = aux
    #역전파를 위한 G_OUTPUT구하기 \ G_OUTPUT0는 3개, G_OUTPUT1은 31개
    G_ouput0 = self.base.backprop_postproc(G_loss, aux0, 'select')
    G_ouput1 = self.base.backprop_postproc(G_loss, aux1, 'select')

    return np.hstack([G_ouput0, G_ouput1])

In [10]:
def office31_eval_accuracy(self, x, y, output):
    #예측 output과 실제값 y를 받아 각각 분할
    outputs, ys = np.hsplit(output, self.cnts), np.hsplit(y, self.cnts)

    #값을 전달받아 dataset에 있는 eval_accuracy()에 값을 전달하여 acc 구하기
    acc0 = self.base.eval_accuracy(x, ys[0], outputs[0], 'select')
    acc1 = self.base.eval_accuracy(x, ys[1], outputs[1], 'select')

    return [acc0, acc1]

In [12]:
def office31_train_prt_result(self, epoch, costs, accs, acc, time1, time2):
    acc_pair = np.mean(accs, axis=0)
    print('Epoch {}: cost={:5.3f}, ' \
          'accuracy={:5.3f}+{:5.3f}/{:5.3f}+{:5.3f} ({}/{} secs)'. \
          format(epoch, np.mean(costs), acc_pair[0], acc_pair[1], \
                 acc[0], acc[1], time1, time2))

def office31_test_prt_result(self, name, acc, time):
    print('Model {} test report: accuracy = {:5.3f}+{:5.3f}, ({} secs)\n'. \
          format(name, acc[0], acc[1], time))
